In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv2D,MaxPooling2D,Dropout

In [2]:
gen=ImageDataGenerator(rescale=1/255.0,rotation_range=90,vertical_flip=True,horizontal_flip=True)
train_itr=gen.flow_from_directory("./gender_training/gender_training/train",target_size=(90,90),color_mode='grayscale',batch_size=32,class_mode='binary')
val_itr=gen.flow_from_directory("./gender_training/gender_training/val",target_size=(90,90),color_mode='grayscale',batch_size=32,class_mode='binary')
test_itr=gen.flow_from_directory("./gender_training/gender_training/test",target_size=(90,90),color_mode='grayscale',batch_size=32,class_mode='binary')

Found 300 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [3]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(90,90,1)))
model.add(MaxPooling2D())

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D())


model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))

model.add(Dense(2,activation="softmax"))
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=['accuracy'])


In [7]:
model.fit(test_itr,epochs=40,validation_data=val_itr)

Epoch 1/40
2/2 [==============================] - 0s 170ms/step - loss: 0.4115 - accuracy: 0.8167 - val_loss: 0.6182 - val_accuracy: 0.7500
Epoch 2/40
2/2 [==============================] - 0s 138ms/step - loss: 0.3301 - accuracy: 0.8333 - val_loss: 0.5250 - val_accuracy: 0.7250
Epoch 3/40
2/2 [==============================] - 0s 144ms/step - loss: 0.3749 - accuracy: 0.8167 - val_loss: 0.5280 - val_accuracy: 0.7750
Epoch 4/40
2/2 [==============================] - 0s 157ms/step - loss: 0.3980 - accuracy: 0.8167 - val_loss: 0.5257 - val_accuracy: 0.7000
Epoch 5/40
2/2 [==============================] - 0s 140ms/step - loss: 0.3127 - accuracy: 0.8833 - val_loss: 0.5476 - val_accuracy: 0.7250
Epoch 6/40
2/2 [==============================] - 0s 144ms/step - loss: 0.3297 - accuracy: 0.8333 - val_loss: 0.5602 - val_accuracy: 0.7750
Epoch 7/40
2/2 [==============================] - 0s 138ms/step - loss: 0.2857 - accuracy: 0.8833 - val_loss: 0.4663 - val_accuracy: 0.8250
Epoch 8/40
2/2 [====

In [10]:
model.evaluate(test_itr,steps=10)

 2/10 [=====>........................] - 0s 40ms/step - loss: 0.2901 - accuracy: 0.8500


[0.2900760769844055, 0.8500000238418579]

In [11]:
train_itr.class_indices

{'female': 0, 'male': 1}

In [14]:
import cv2
import numpy as np
img=cv2.imread("../vikas2.JPEG")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#gray=gray/255.0
faceDetect=cv2.CascadeClassifier('f:haarcascade/haarcascade_frontalface_default.xml')
faces = faceDetect.detectMultiScale(gray)
for (x,y,w,h) in faces:
    face=gray[y:y+h,x:x+w]
    face=cv2.resize(face,(90,90))
    print(np.argmax(model.predict(face.reshape(1,90,90,1)),axis=-1))

[1]
[1]
[1]
[1]


In [21]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

face_cascade  = cv2.CascadeClassifier('f:haarcascade/haarcascade_frontalface_default.xml')

while True:
    asdf,img = cap.read()
    grayimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(grayimg,1.1,4)
    
    for x,y,w,h in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),3)
        face=grayimg[y:y+h,x:x+w]
        face=cv2.resize(face,(90,90))
    x = np.argmax(model.predict(face.reshape(1,90,90,1)),axis=-1)
    if x == 1:
        cv2.putText(img,'Male',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
    elif x==0:
        cv2.putText(img,'Female',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
        
    cv2.imshow('Live Video',img)
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break
        
cap.release()
cv2.destroyAllWindows()        

In [17]:
cap.release()